<a href="https://colab.research.google.com/github/AhrazKhan31/Deep-Learning-Lab/blob/main/Experiment3_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from torchvision.datasets import ImageFolder
import os
from collections import defaultdict


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tongpython/cat-and-dog")

print("Path to dataset files:", path)

100%|██████████| 218M/218M [00:08<00:00, 25.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/tongpython/cat-and-dog/versions/1


In [ ]:
# For Cats vs. Dogs dataset
full_data = ImageFolder(root='/root/.cache/kagglehub/datasets/tongpython/cat-and-dog/versions/1')
print(f"Total samples: {len(full_data)}")

# Split into train/val
train_size = int(0.8 * len(full_data))
val_size = len(full_data) - train_size
train_data, val_data = random_split(full_data, [train_size, val_size],
                                  generator=torch.Generator().manual_seed(42))  # Fix seed

print(f"Train samples: {len(train_data)}, Val samples: {len(val_data)}")

Total samples: 10028
Train samples: 8022, Val samples: 2006


In [ ]:
# Define CNN with dynamic output size
class CNN(nn.Module):
    def __init__(self, activation_fn, weight_init, num_classes=2):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            activation_fn(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            activation_fn(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            activation_fn(),
            nn.MaxPool2d(2)
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(128*4*4, 512),
            activation_fn(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

        self.activation = activation_fn
        self._initialize_weights(weight_init)

    def _initialize_weights(self, weight_init):
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                if weight_init == 'xavier':
                    nn.init.xavier_normal_(m.weight)
                elif weight_init == 'kaiming':
                    nn.init.kaiming_normal_(m.weight, mode='fan_in',
                                         nonlinearity='relu' if isinstance(self.activation, nn.ReLU) else 'leaky_relu')
                else:
                    nn.init.normal_(m.weight, 0, 0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        return self.fc_layers(x)


In [ ]:
# Dataset loader with model-specific preprocessing
def load_data(dataset_name, model_type='cnn'):
    # Configure preprocessing
    if model_type == 'cnn':
        resize = 32
        normalize = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    else:  # For ResNet
        resize = 224
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                       std=[0.229, 0.224, 0.225])

    transform = transforms.Compose([
        transforms.Resize((resize, resize)),
        transforms.ToTensor(),
        normalize
    ])
    # Load dataset
    if dataset_name == 'cats_vs_dogs':
        # For Cats vs. Dogs dataset
        full_data = ImageFolder(root='/root/.cache/kagglehub/datasets/tongpython/cat-and-dog/versions/1', transform=transform)
        train_data, val_data = random_split(full_data, [train_size, val_size],
                                  generator=torch.Generator().manual_seed(42))  # Fix seed
        num_classes = 2
        test_data = val_data
    elif dataset_name == 'cifar10':
        train_data = datasets.CIFAR10(root='./data', train=True, download=True,
                                    transform=transform)
        test_data = datasets.CIFAR10(root='./data', train=False, download=True,
                                   transform=transform)
        num_classes = 10

    return DataLoader(train_data, 64, True), DataLoader(test_data, 64, False), num_classes


In [ ]:
def train_and_validate(model, train_loader, test_loader, criterion, optimizer, device, epochs=1, dataset_name=None):
    best_acc = 0.0
    history = defaultdict(list)

    for epoch in range(epochs):
        # Training phase
        model.train()
        train_correct = 0
        train_total = 0
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

        train_acc = train_correct / train_total  # Training accuracy

        # Validation phase
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

                # Debugging: Print predictions and labels
                if val_total < 10:  # Print first 10 samples
                   print(f'Predicted: {predicted}')
                   print(f'Labels: {labels}')

        val_acc = val_correct / val_total  # Validation accuracy
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        history['loss'].append(running_loss / len(train_loader))

        # Save best model
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), f'best_{model.__class__.__name__}_{dataset_name}.pth')

        print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}, '
              f'Train Acc: {train_acc:.2%}, Val Acc: {val_acc:.2%}')

    return best_acc, history

In [ ]:
# Experimentation framework
def run_experiments(dataset_name):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    train_loader, test_loader, num_classes = load_data(dataset_name, 'cnn')

    best_overall = {'accuracy': 0, 'config': None}

    # Hyperparameter combinations
    for activation in [nn.ReLU, nn.Tanh, nn.LeakyReLU]:
        for init in ['xavier', 'kaiming', 'random']:
            for opt in [optim.SGD, optim.Adam, optim.RMSprop]:
                print(f'\nTraining with {activation.__name__}, {init}, {opt.__name__}')

                model = CNN(activation, init, num_classes).to(device)
                optimizer = opt(model.parameters(), lr=0.001)
                criterion = nn.CrossEntropyLoss()

                best_acc, _ = train_and_validate(model, train_loader, test_loader,
                                               criterion, optimizer, device, dataset_name=dataset_name)

                # Compare and update best_overall with detailed information
                if best_acc > best_overall['accuracy'] + 1e-4:
                    best_overall['accuracy'] = best_acc
                    best_overall['config'] = {
                        'activation': activation.__name__,
                        'init': init,
                        'optimizer': opt.__name__,
                        'accuracy': best_acc  # Store accuracy in config
                    }

    print(f'\nBest configuration: {best_overall["config"]} with accuracy {best_overall["accuracy"]:.2%}')


In [ ]:
# ResNet-18 fine-tuning
def run_resnet(dataset_name):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    train_loader, test_loader, num_classes = load_data(dataset_name, 'resnet')

    model = models.resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    model = model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    best_acc, _ = train_and_validate(model, train_loader, test_loader,
                                   criterion, optimizer, device)

    print(f'ResNet-18 achieved {best_acc:.2%} accuracy on {dataset_name}')
    torch.save(model.state_dict(), f'best_ResNet18_{dataset_name}.pth')


In [ ]:
if __name__ == '__main__':
    for dataset in ['cats_vs_dogs', 'cifar10']:
        print(f'\n{"="*40}\nRunning experiments on {dataset}\n{"="*40}')
        run_experiments(dataset)


Running experiments on cats_vs_dogs

Training with ReLU, xavier, SGD
Epoch 1/1, Loss: 0.7685, Train Acc: 71.24%, Val Acc: 78.86%

Training with ReLU, xavier, Adam
Epoch 1/1, Loss: 0.6814, Train Acc: 78.09%, Val Acc: 79.31%

Training with ReLU, xavier, RMSprop
Epoch 1/1, Loss: 1.6303, Train Acc: 75.90%, Val Acc: 78.12%

Training with ReLU, kaiming, SGD
Epoch 1/1, Loss: 0.8138, Train Acc: 70.99%, Val Acc: 78.91%

Training with ReLU, kaiming, Adam
Epoch 1/1, Loss: 0.8847, Train Acc: 77.27%, Val Acc: 79.31%

Training with ReLU, kaiming, RMSprop
Epoch 1/1, Loss: 1.4673, Train Acc: 76.27%, Val Acc: 79.31%

Training with ReLU, random, SGD
Epoch 1/1, Loss: 0.5241, Train Acc: 79.62%, Val Acc: 79.31%

Training with ReLU, random, Adam
Epoch 1/1, Loss: 0.5226, Train Acc: 79.89%, Val Acc: 79.31%

Training with ReLU, random, RMSprop
Epoch 1/1, Loss: 0.8903, Train Acc: 78.09%, Val Acc: 79.31%

Training with Tanh, xavier, SGD
Epoch 1/1, Loss: 0.6342, Train Acc: 73.50%, Val Acc: 79.06%

Training with 

100%|██████████| 170M/170M [00:01<00:00, 104MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified

Training with ReLU, xavier, SGD
Epoch 1/1, Loss: 2.0606, Train Acc: 28.62%, Val Acc: 42.16%

Training with ReLU, xavier, Adam
Epoch 1/1, Loss: 1.5068, Train Acc: 46.10%, Val Acc: 60.01%

Training with ReLU, xavier, RMSprop
Epoch 1/1, Loss: 1.8121, Train Acc: 40.35%, Val Acc: 45.62%

Training with ReLU, kaiming, SGD
Epoch 1/1, Loss: 2.2150, Train Acc: 24.84%, Val Acc: 39.24%

Training with ReLU, kaiming, Adam
Epoch 1/1, Loss: 1.5330, Train Acc: 45.04%, Val Acc: 61.16%

Training with ReLU, kaiming, RMSprop
Epoch 1/1, Loss: 1.7694, Train Acc: 42.08%, Val Acc: 55.12%

Training with ReLU, random, SGD
Epoch 1/1, Loss: 2.0562, Train Acc: 27.35%, Val Acc: 39.59%

Training with ReLU, random, Adam
Epoch 1/1, Loss: 1.3647, Train Acc: 50.24%, Val Acc: 61.02%

Training with ReLU, random, RMSprop
Epoch 1/1, Loss: 1.6199, Train Acc: 41.85%, Val Acc: 47.07%

Training with Tanh, xavier, SGD
Epoch 1/1, Loss: 2.0610

In [ ]:
if __name__ == '__main__':
    for dataset in ['cats_vs_dogs', 'cifar10']:
        print(f'\n{"="*40}\nRunning experiments on {dataset}\n{"="*40}')
        run_resnet(dataset)


Running experiments on cats_vs_dogs


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 162MB/s]


Epoch 1/1, Loss: 0.5538, Train Acc: 78.42%, Val Acc: 78.86%
ResNet-18 achieved 78.86% accuracy on cats_vs_dogs

Running experiments on cifar10


100%|██████████| 170M/170M [00:11<00:00, 14.7MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [9]:
if __name__ == '__main__':
    for dataset in ['cifar10']:
        print(f'\n{"="*40}\nRunning experiments on {dataset}\n{"="*40}')
        run_resnet(dataset)


Running experiments on cifar10


100%|██████████| 170M/170M [00:11<00:00, 14.5MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 103MB/s]


Epoch 1/1, Loss: 0.5561, Train Acc: 81.00%, Val Acc: 84.50%
ResNet-18 achieved 84.50% accuracy on cifar10
